## Regras de Seleção do Dataset

Foram incluídos no dataset apenas documentos oficiais do restaurante,
compostos por fichas técnicas de receitas (PDFs) e imagens de apoio
ao preparo e empratamento.

Foram excluídos documentos externos, materiais promocionais e arquivos
sem relação direta com o processo culinário padronizado.

In [11]:
import pandas as pd
from pathlib import Path

# Caminho base do projeto
BASE_PATH = Path("..") / "dataset_restaurante"

# Caminho do CSV
CSV_PATH = BASE_PATH / "inventario_dataset.csv"

df = pd.read_csv(CSV_PATH)
df.head()


,document_id,tipo,path_arquivo,titulo,origem,data,categoria,versao,nivel_confianca
0,PDF_001,pdf,fichas_tecnicas/ficha_01_baiao_de_dois.pdf,Baiao-de-Dois,Ficha técnica oficial,2025-02-05,Tradicionais,v1.0,alto
1,PDF_002,pdf,fichas_tecnicas/ficha_02_favada.pdf,Favada,Ficha técnica oficial,2025-12-04,Tradicionais,v1.0,alto
2,PDF_003,pdf,fichas_tecnicas/ficha_03_feijao_de_corda.pdf,Feijao-de-Corda,Ficha técnica oficial,2025-05-31,Tradicionais,v1.0,alto
3,PDF_004,pdf,fichas_tecnicas/ficha_04_sarapatel.pdf,Sarapatel,Ficha técnica oficial,15/03/2025,Tradicionais,v1.0,alto
4,PDF_005,pdf,fichas_tecnicas/ficha_05_caldo_de_mocoto.pdf,Caldo de Mocoto,Ficha técnica oficial,11-05-2025,Tradicionais,v1.0,alto


In [12]:
def arquivo_existe(path_relativo):
    return (BASE_PATH / path_relativo).exists()

df["arquivo_existe"] = df["path_arquivo"].apply(arquivo_existe)

df[~df["arquivo_existe"]]


,document_id,tipo,path_arquivo,titulo,origem,data,categoria,versao,nivel_confianca,arquivo_existe
20,PDF_021,pdf,imagens/arquivo_nao_existe.jpg,Creme Brulee de Doce de Leite,Ficha técnica oficial,08-03-2025,Sobremesas,v1.0,alto,False
37,IMG_012,imagem,imagens/IMG_12.jpg,Empratamento - Rabada,Manual interno do restaurante,2025-07-25,Especialidades,v1.0,alto,False
47,IMG_022,imagem,imagens/img_22pratofinal.jpg,Empratamento - Cocada Cremosa,Manual interno do restaurante,24/07/2025,Sobremesas,v1.0,alto,False


In [13]:
def tipo_valido(row):
    if row["tipo"] == "pdf":
        return row["path_arquivo"].endswith(".pdf")
    if row["tipo"] == "imagem":
        return row["path_arquivo"].lower().endswith((".jpg", ".png", ".jpeg"))
    return False

df["tipo_valido"] = df.apply(tipo_valido, axis=1)

df[~df["tipo_valido"]]


,document_id,tipo,path_arquivo,titulo,origem,data,categoria,versao,nivel_confianca,arquivo_existe,tipo_valido
20,PDF_021,pdf,imagens/arquivo_nao_existe.jpg,Creme Brulee de Doce de Leite,Ficha técnica oficial,08-03-2025,Sobremesas,v1.0,alto,False,False


In [14]:
df[df.duplicated(subset=["path_arquivo"], keep=False)]

,document_id,tipo,path_arquivo,titulo,origem,data,categoria,versao,nivel_confianca,arquivo_existe,tipo_valido
5,PDF_006,pdf,fichas_tecnicas/ficha_06_mocofava.pdf,Mocofava,Ficha técnica oficial,15-03-2025,Especialidades,v1.0,alto,True,True
15,PDF_016,pdf,fichas_tecnicas/ficha_16_vinagrete_sertanejo.pdf,Vinagrete Sertanejo,Ficha técnica oficial,15/03/2025,Saladas,v1.0,alto,True,True
52,PDF_006,pdf,fichas_tecnicas/ficha_06_mocofava.pdf,Mocofava,Ficha técnica oficial,15-03-2025,Especialidades,v1.0,alto,True,True
53,PDF_016,pdf,fichas_tecnicas/ficha_16_vinagrete_sertanejo.pdf,Vinagrete Sertanejo,Ficha técnica oficial,15/03/2025,Saladas,v1.0,alto,True,True


In [15]:
import re

def nome_imagem_valido(path):
    nome = path.split("/")[-1]
    padrao = r"^img_\d{2}_.+\.jpg$"
    return bool(re.match(padrao, nome))
df["nome_padrao_valido"] = df["path_arquivo"].apply(
    lambda x: nome_imagem_valido(x) if str(x).endswith(".jpg") else True
)
df_erros_nome = df[
    (df["tipo"] == "imagem") &
    (df["nome_padrao_valido"] == False)
][["document_id", "path_arquivo", "titulo"]]

df_erros_nome


,document_id,path_arquivo,titulo
37,IMG_012,imagens/IMG_12.jpg,Empratamento - Rabada
47,IMG_022,imagens/img_22pratofinal.jpg,Empratamento - Cocada Cremosa


PDF com OCR simulado

O documento PDF_013 – costelinha-de-porco-recheada foi definido propositalmente como exemplo de PDF escaneado (OCR simulado).

Apesar de conter texto extraível, esse tipo de documento representa limitações comuns em pipelines de ingestão de dados, como estrutura irregular, perda de semântica e dificuldade de segmentação adequada para RAG.

A identificação foi realizada manualmente para fins didáticos.

In [16]:
# Marcação manual de PDF com OCR simulado (apenas para análise)
df["ocr_simulado"] = False
df.loc[df["document_id"] == "PDF_013", "ocr_simulado"] = True

df[df["ocr_simulado"]]


,document_id,tipo,path_arquivo,titulo,origem,data,categoria,versao,nivel_confianca,arquivo_existe,tipo_valido,nome_padrao_valido,ocr_simulado
12,PDF_013,pdf,fichas_tecnicas/ficha_13_costelinha_de_porco_r...,Costelinha de Porco Recheada,Ficha técnica oficial,2025-07-10,Especialidades,v1.0,alto,True,True,True,True


In [17]:
df_validos = df[
    (df["arquivo_existe"] == True) &
    (df["tipo_valido"] == True)
]


In [18]:
DOCUMENTOS_INVALIDOS = {"PDF_021", "IMG_012", "IMG_022"}
df_validos = df[~df["document_id"].isin(DOCUMENTOS_INVALIDOS)]


In [19]:
df_validos["document_id"].unique()


array(['PDF_001', 'PDF_002', 'PDF_003', 'PDF_004', 'PDF_005', 'PDF_006',
       'PDF_007', 'PDF_008', 'PDF_009', 'PDF_010', 'PDF_011', 'PDF_012',
       'PDF_013', 'PDF_014', 'PDF_015', 'PDF_016', 'PDF_017', 'PDF_018',
       'PDF_019', 'PDF_020', 'PDF_022', 'PDF_023', 'PDF_024', 'PDF_025',
       'PDF_026', 'IMG_001', 'IMG_002', 'IMG_003', 'IMG_004', 'IMG_005',
       'IMG_006', 'IMG_007', 'IMG_008', 'IMG_009', 'IMG_010', 'IMG_011',
       'IMG_013', 'IMG_014', 'IMG_015', 'IMG_016', 'IMG_017', 'IMG_018',
       'IMG_019', 'IMG_020', 'IMG_021', 'IMG_023', 'IMG_024', 'IMG_025',
       'IMG_026'], dtype=object)

In [20]:
print("Mini-inventário do Dataset")
print("PDFs:", df[df["tipo"] == "pdf"]["document_id"].nunique())
print("Imagens:", df[df["tipo"] == "imagem"]["document_id"].nunique())
print("Linhas no CSV:", len(df))
print("Válidos:", len(df_validos))
print("Inválidos:", len(df) - len(df_validos))

Mini-inventário do Dataset
PDFs: 26
Imagens: 26
Linhas no CSV: 54
Válidos: 51
Inválidos: 3


## Mini-inventário do Dataset

- PDFs: 26  
- Imagens: 26  
- Linhas no CSV: 54  

Observação: o total de linhas no CSV inclui **duplicados e registros inválidos intencionais**,
mantidos nesta etapa para fins de auditoria e validação.